In [1]:
%matplotlib inline

import re

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

        
def map_array(map_txt):
    rows = map_txt.splitlines()
    map_ = np.array([list(r) for r in rows])
    return map_


def print_map(map_):
    for row in map_:
        print(''.join(row))

In [2]:
test = """
O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#....
""".strip()

In [3]:
with open('input.txt', 'r') as f:
    input_ = f.read().strip()

# Part 1

In [13]:
map_ = map_array(test)

In [17]:
np.where(map_ == '#')

(array([0, 1, 1, 2, 2, 3, 4, 5, 5, 5, 6, 8, 8, 8, 8, 9, 9]),
 array([5, 4, 9, 5, 6, 3, 8, 2, 7, 9, 5, 0, 5, 6, 7, 0, 5]))

In [93]:
def northen_square(map_):
    h = map_.shape[0]
    last_square_row = np.zeros_like(map_, dtype=int)
    for r in range(h):
        if r > 0:
            last_square_row[r,:] = last_square_row[r-1,:]
        squares = np.nonzero(map_[r,:] == '#')
        last_square_row[r, squares] = r + 1
    return last_square_row

In [94]:
# map with the row index of the northmost square per column (cumulative)
map_ = map_array(test)
h = map_.shape[0]
cr, cc = np.where(map_ == 'O')
last_square_row = northen_square(map_)
tot = 0
for r, c in zip(cr, cc):
    last_square_r = last_square_row[r, c]
    weight = h - last_square_r - (map_[last_square_r:r, c] == 'O').sum()
    tot += weight
tot

136

In [95]:
# map with the row index of the northmost square per column (cumulative)
map_ = map_array(input_)
h = map_.shape[0]
cr, cc = np.where(map_ == 'O')
last_square_row = northen_square(map_)
tot = 0
for r, c in zip(cr, cc):
    last_square_r = last_square_row[r, c]
    weight = h - last_square_r - (map_[last_square_r:r, c] == 'O').sum()
    tot += weight
tot

103333

# Part 2

In [4]:
def new_map(map_, circle_pos):
    square_map = map_.copy()
    square_map[square_map == 'O'] = '.'
    for r, c in circle_pos:
        square_map[r, c] = 'O'
    return square_map

In [5]:
# Rolling maps

def northern_square(map_):
    h = map_.shape[0]
    last_square_row = np.zeros_like(map_, dtype=int)
    for r in range(h):
        if r > 0:
            last_square_row[r,:] = last_square_row[r-1,:]
        squares = np.nonzero(map_[r,:] == '#')
        last_square_row[r, squares] = r + 1
    return last_square_row


def southern_square(map_):
    h = map_.shape[0]
    last_square_row = np.zeros_like(map_, dtype=int) + h - 1
    for r in range(h-1, -1, -1):
        if r < h-1:
            last_square_row[r,:] = last_square_row[r+1,:]
        squares = np.nonzero(map_[r,:] == '#')
        last_square_row[r, squares] = r - 1
    return last_square_row


def eastern_square(map_):
    w = map_.shape[1]
    last_square_col = np.zeros_like(map_, dtype=int) + w - 1
    for c in range(w-1, -1, -1):
        if c < w-1:
            last_square_col[:,c] = last_square_col[:,c+1]
        squares = np.nonzero(map_[:, c] == '#')
        last_square_col[squares, c] = c - 1
    return last_square_col


def western_square(map_):
    w = map_.shape[1]
    last_square_col = np.zeros_like(map_, dtype=int)
    for c in range(w):
        if c > 0:
            last_square_col[:,c] = last_square_col[:,c-1]
        squares = np.nonzero(map_[:, c] == '#')
        last_square_col[squares, c] = c + 1
    return last_square_col

In [6]:
map_orig = map_array(input_)
h = map_orig.shape[0]
w = map_orig.shape[1]

north_square_row = northern_square(map_orig)
east_square_col = eastern_square(map_orig)
south_square_row = southern_square(map_orig)
west_square_col = western_square(map_orig)
circle_pos = list(zip(*np.where(map_orig == 'O')))

map_ = new_map(map_orig, circle_pos)
last_map = map_.copy()
map_history = []
pos_history = []

n_cycles = 0
while n_cycles < 1:
    # roll north
    new_circle_pos = []
    for r, c in circle_pos:
        new_c = c
        square_r = north_square_row[r, c]
        new_r = square_r + (map_[square_r:r, c] == 'O').sum()
        new_circle_pos.append((new_r, new_c))

    circle_pos = new_circle_pos
    map_ = new_map(map_orig, circle_pos)
    #print_map(map_)
    #print((map_ == 'O').sum())

    # roll west
    new_circle_pos = []
    for r, c in circle_pos:
        new_r = r
        square_c = west_square_col[r, c]
        new_c = square_c + (map_[r, square_c:c] == 'O').sum()
        new_circle_pos.append((new_r, new_c))

    circle_pos = new_circle_pos
    map_ = new_map(map_orig, circle_pos)
    #print_map(map_)
    #print((map_ == 'O').sum())

    # roll south
    new_circle_pos = []
    for r, c in circle_pos:
        new_c = c
        square_r = south_square_row[r, c]
        new_r = square_r - (map_[r+1:square_r+1, c] == 'O').sum()
        new_circle_pos.append((new_r, new_c))

    circle_pos = new_circle_pos
    map_ = new_map(map_orig, circle_pos)
    #print_map(map_)
    #print((map_ == 'O').sum())

    # roll east
    new_circle_pos = []
    for r, c in circle_pos:
        new_r = r
        square_c = east_square_col[r, c]
        new_c = square_c - (map_[r, c+1:square_c+1] == 'O').sum()
        new_circle_pos.append((new_r, new_c))

    circle_pos = new_circle_pos
    map_ = new_map(map_orig, circle_pos)
    #print_map(map_)
    #print((map_ == 'O').sum())

    # Detect cycle
    n_cycles = 0
    for m in map_history:
        if np.all(map_ == m):
            n_cycles += 1
    
    map_history.append(map_)
    pos_history.append(circle_pos)


In [16]:
start_idx = -1
for idx, n in enumerate(map_history[:-1]):
    is_cycle = np.all(map_history[-1] == n)
    if is_cycle:
        start_idx = idx

In [17]:
cycle_len = len(map_history) - start_idx - 1
cycle_len

36

In [18]:
(start_idx + 1)

99

In [19]:
far_away_idx = (1000000000 - (start_idx + 1)) % (len(map_history) - start_idx - 1)
far_away_idx

1

In [20]:
tot = 0
for r, c in pos_history[start_idx + far_away_idx]:
    weight = h - r
    tot += weight
tot

97241